In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

In [2]:
import matplotlib.pyplot as plt, argparse, numpy as np, math, sys, copy
from skimage.segmentation import slic, mark_boundaries, felzenszwalb
from skimage.util import img_as_float
from skimage import io
import numpy as np
import superpixels as sp
import glob
import pipeline as pipe
import time
import superpixels as sp
import scipy.stats as sci
from PIL import Image
from scipy import ndimage
import cv2

In [3]:
ground_path = 'BSDS500/data/groundTruth/'
data_path = 'BSDS500/data/images/'
save_path = 'BSDS500/pre_process/images/'
bound_prefix = 'images/'
eval_prefix = 'eval/'

epsilon = 0.0001 #just to round down
DEFAULT_IMG_SIZE = (481,481,3) #defaul image size

### Read Images

In [5]:
data_types = ['val', 'train']

image_names, names = [], []

for data_type in data_types:
    image_path = data_path + data_type + '/'
    names_path = save_path + data_type + '/'
    
    len_path = len(image_path)
    image_files = glob.glob(image_path + '*.jpg')
    image_filenames = [x[len_path:-4] for x in image_files]

    #load images
    for filename in image_filenames:
        file = image_path + filename + '.jpg'

        #append image and groundtruths
        image_names.append(file)
        names.append(names_path + filename + '_&&&.png')

print(len(image_names), len(names))

300 300


In [10]:
index = 0
len_images = len(names)
for image_name, name in zip(image_names, names):
    index += 1

    black_image = np.zeros(DEFAULT_IMG_SIZE,dtype=np.uint8)
    image = io.imread(image_name)
    
    prefix_name = name[:-4]
    extension_name = name[-4:]

    if(image.shape[0] == 321):
        black_image[80:401,:,:] = image[:,:,:]
    else:
        black_image[:,80:401,:] = image[:,:,:]
        
    flip_types = ['noflip', 'fliplr']
    
    for flip_type in flip_types:
        if flip_type == 'fliplr':
            flip_image = np.fliplr(image)
            flip_black = np.fliplr(black_image)
        elif flip_type == 'flipud':
            flip_image = np.flipud(image)
            flip_black = np.flipud(black_image)
        else:
            flip_black = black_image
            flip_image = image
        
        #save image 0
        save_name = prefix_name.replace("&&&", ('_' + flip_type + '_0'))
        save_name = (save_name + extension_name)
        io.imsave(save_name, flip_black)

        #rotate image and save
        for angle in range(30, 360, 30):
            rot_image = ndimage.rotate(flip_image, angle)
            res_image = cv2.resize(rot_image, dsize=DEFAULT_IMG_SIZE[:2], interpolation=cv2.INTER_CUBIC)

            save_name = prefix_name.replace("&&&", ('_' + flip_type + '_' + str(angle)))
            save_name = (save_name + extension_name)
            io.imsave(save_name, res_image)
    
    print(index, '/', len_images, end='')
    print('\r', end='')

print('OK!')

OK! / 300


### Generate boundaries and save results

In [ ]:
ini = 0
fim = len(ground_filenames)

methods = ['hsgb'] #['hslic', 'hsgb', 'hegb']

for method in methods:
    last_shape = None
    blank_image = None
    index = 0
    
    starttime = time.time()
    for image, name in zip(images[ini:fim], names[ini:fim]):
        if(image.shape != last_shape):
            blank_image = np.zeros(image.shape,dtype=np.uint8) #create blank image to save
            blank_image.fill(255)
            last_shape = image.shape
        
        ##generate ultrametric map
        ultra_image, cluster_sizes = pipe.generate_ultrametric_image(image, blank_image, method)
        
        #convert to grayscale
        gray_image = np.dot(ultra_image[...,:3], [0.299, 0.587, 0.114])
        
        #save image
        boundname = save_path + method + '/' + bound_prefix + name
        io.imsave(boundname, gray_image)
        
        #verbose mode
        index += 1
        if index % 10 == 0:
            print(index, cluster_sizes)

    endtime = time.time()
        
    #np.savez(boundname, borders=borders)
    print((endtime - starttime), 'seconds')

### Recover boundaries and evaluate results

In [ ]:
ini = 0
fim = len(ground_filenames)

methods = ['hsgb'] #['hslic', 'hsegb', 'hegb']

for method in methods:    
    threshold = 0
    index = 0

    starttime = time.time()
    
    for groundtruth, name in zip(groundtruths[ini:fim], names[ini:fim]):
        precisions, recalls = [], []
        
        boundname = save_path + method + '/' + bound_prefix + name
        evalname = save_path + method + '/' + eval_prefix + name + extension
        
        image = img_as_float(io.imread(boundname))        
        unique = np.unique(image)
        
        #adjust number of clusters (only to some itens)
        if(len(unique) <= number_clusters):
            len_unique = len(unique)
            for i in range(number_clusters-len_unique+1):
                unique = np.append(unique, unique[len_unique-1])
        
        for thresh in unique[1:]:
            thresh_image = sci.threshold(image, (thresh - epsilon))

            #calculate values from precision and recall
            precision, recall = ev.eval_bound(groundtruth[:,:,0], thresh_image, threshold, True)
            precisions.append(precision)
            recalls.append(recall)
        
        #verbose mode
        index += 1
        if index % 10 == 0:
            print(index)
        
        #save precision file
        np.savez(evalname, precisions=precisions, recalls=recalls)
            
    endtime = time.time()
    print((endtime - starttime), 'seconds')

### Load evaluations and calculate f-measures

In [ ]:
ini = 0
fim = len(ground_filenames)

methods = ['hsgb'] #['hslic', 'hsegb', 'hegb']

for method in methods:
    precisions, recalls = [], []
    fmeasures = []
    
    for name in names[ini:fim]:
        evalname = save_path + method + '/' + eval_prefix + name + extension
        npload = np.load(evalname)
        precisions.append(npload['precisions'])
        recalls.append(npload['recalls'])

    for i in range(0,len(precisions[0])):
        cutz_precision, cutz_recall = [], []
        
        for precision, recall in zip(precisions, recalls):
            cutz_precision.append(precision[i])
            cutz_recall.append(recall[i])
    
        np_precisions = (np.array(cutz_precision))
        np_recalls = (np.array(cutz_recall))

        fmeasure = 2 * ((np_precisions * np_recalls) / (np_precisions + np_recalls))
        fmeasures.append(fmeasure)
    
    #print(fmeasures)
print(np.average(fmeasures, axis=1))